In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms, models, datasets
from PIL import Image
from torch import optim
from glob import glob
import cv2, glob, numpy as np, pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F


In [5]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json
kaggle.json


In [7]:
!kaggle datasets download -d tongpython/cat-and-dog
!unzip cat-and-dog.zip

Streaming output truncated to the last 5000 lines.
  inflating: training_set/training_set/cats/cat.3704.jpg  
  inflating: training_set/training_set/cats/cat.3705.jpg  
  inflating: training_set/training_set/cats/cat.3706.jpg  
  inflating: training_set/training_set/cats/cat.3707.jpg  
  inflating: training_set/training_set/cats/cat.3708.jpg  
  inflating: training_set/training_set/cats/cat.3709.jpg  
  inflating: training_set/training_set/cats/cat.371.jpg  
  inflating: training_set/training_set/cats/cat.3710.jpg  
  inflating: training_set/training_set/cats/cat.3711.jpg  
  inflating: training_set/training_set/cats/cat.3712.jpg  
  inflating: training_set/training_set/cats/cat.3713.jpg  
  inflating: training_set/training_set/cats/cat.3714.jpg  
  inflating: training_set/training_set/cats/cat.3715.jpg  
  inflating: training_set/training_set/cats/cat.3716.jpg  
  inflating: training_set/training_set/cats/cat.3717.jpg  
  inflating: training_set/training_set/cats/cat.3718.jpg  
  infl

In [8]:
#@ Specifying the training and test image folder:
train_data_dir='training_set/training_set'
test_data_dir= 'test_set/test_set'

In [9]:
class CatsDogs(Dataset):
  def __init__(self, folder):
    cats=glob(folder+'/cats/*.jpg')
    dogs=glob(folder+'dogs/*.jpg')
    self.fpaths=cats[:500] + dogs[:500]
    self.normalize=transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    from random import shuffle, seed; seed(10);
    shuffle(self.fpaths)
    self.targets=[fpath.split('/')[-1].startswith('dog') for fpath in self.fpaths]


  def __len__(self):
    return len(self.fpaths)

  def __getitem__(self, ix):
    f=self.fpaths[ix]
    target=self.targets[ix]
    im=(cv2.imread(f)[:, :, ::-1])
    im=cv2.resize(im, (224, 224))
    im=torch.tensor(im/255)
    im=im.permute(2, 0, 1)
    im=self.normalize(im)
    return ( im.float().to(device),
             torch.tensor([target]).float().to(device))